# Laboratory work #2

Comparison of the performance of algorithms on certain text data of different sizes

# 1. Huffman's algorithm

Алгоритм Гаффмана
Клас містить два методи:

encode: закодовує повідомлення
Спочатку створює словник з різних символів і рахує, скільки разів кожен з них зустрічався в повідомленні. Далі на основі цього рахуються ймовірності, з якими зустрічається кожен символ. Потім з цього словника робимо ліст тюплів. І сортуємо його так, щоб спочатку були символи з найменшою ймовірністю. Після цього об'єднюємо перші два елементи в один, додаємо '1' та '0' до їх кодів відповідно (0 – до того, що більше), видаляємо ці елементи зі списку, додаємо туди новий елемент(його ймовірність – сума попередніх двох) і знову сортуємо наш список. Повторюємо це, поки не залишиться один елемент в списку. Тоді у нас вже є в словник кодів. Закодовуємо кожен символ повідомлення та повертаємо результат.

decode: розкодовує повідомлення
Маючи словник кодів розкодовуємо закодоване повідомлення.
Считуємо символ, якщо він не відповідає жодному коду в словнику, считуємо наступний. Коли код є в словнику, додаємо розкодований символ до розкодованого повідомлення. І обрізаємо закодоване повідомлення до першого ще не прочитаного символу. Повторюємо все це, поки не дійдемо до кінця і не розкодуємо все повідомлення, після цього повертаємо результат.

In [ ]:
"""
This module implements Huffman algorithm
"""
from time import time
class Huffman:
    """
    this class allows to use Huffman algorithm for encoding and decoding
    """
    def __init__(self, message:str, res_dict={})-> None:
        """
        message - message, that should be encoded or decoded
        res_dict - dictionary with keys for encoding and decoding
        """
        self.message = message
        self.res_dict = res_dict
    def encode(self, message):
        """
        encode the message using Huffman algorithm
        """
        self.message = message
        freq_dict = {}
        length = len(self.message)

        for elem in self.message:
            if elem not in freq_dict:
                freq_dict[elem] = 1
            else:
                freq_dict[elem] += 1

        for elem in freq_dict:
            freq_dict[elem] /= length
            self.res_dict[elem] = ''
        alphabet = sorted(list(freq_dict.items()), key=lambda x:x[1])
        if len(alphabet) == 1:
            self.res_dict[alphabet[0][0]] = '1'
        else:
            while len(alphabet) > 1:
                new_el = alphabet[0][0] + alphabet[1][0]
                new_val = alphabet[0][1] + alphabet[1][1]
                for i in alphabet[0][0]:
                    self.res_dict[i] = '1' + self.res_dict[i]

                for i in alphabet[1][0]:
                    self.res_dict[i] = '0' + self.res_dict[i]

                alphabet = alphabet[2:]
                alphabet.append((new_el, new_val))
                alphabet = sorted(alphabet, key=lambda x: x[1])

        encoded_mes = ''
        for elem in self.message:
            encoded_mes += self.res_dict[elem]
        return encoded_mes


    def decode(self, message):
        """
        decode the message using Huffman algorithm
        """
       # message = self.message
        dictionary = {v: k for k, v in self.res_dict.items()}
        decoded_mes = ''
        i = 1

        while (message != '') and (i <= len(message)):
            if message[:i] in dictionary:
                decoded_mes += dictionary[message[:i]]
                message = message[i:]
                i = 1
            else:
                i += 1
        return decoded_mes

if __name__ == "__main__":
    with open('text_example.txt', 'r', encoding='utf-8') as file:
        text = ''.join(file.readlines())
        huffman = Huffman(text)
        # measure the compression time
        start = time()
        encoded_message = huffman.encode(text)
        print(f"Time of compress with a file size of {len(text)} characters:  \t\
        {time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")


        # measure the decompression time
        start = time()
        decoded_message = huffman.decode(encoded_message)
        print(f"Time of decompress with a file size of {len(text)} characters:\t\
        {time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")


        # print the percentage of file compression    
        compr = (1 - (len(encoded_message) / (len(text.encode("utf-8"))*8))) * 100
      #  print (f"compression: {round(compr, 2)}%")
        print(f'Percentage of compression with a file size of {len(text)} characters:\
\t{compr}%')
        assert decoded_message == text


# 2. LZW

Цей код реалізує алгоритм LZW для кодування та декодування повідомлень. 
Клас LZW має метод encode, який приймає повідомлення та повертає закодоване повідомлення як список цілих чисел(які відповідають кодам в словнику). Метод decode приймає закодоване повідомлення та повертає розкодоване повідомлення у вигляді рядка.

У процесі кодування створюється словник усіх символів у повідомленні, призначаються їм коди (починаючи з 0), а потім сканується повідомлення, щоб знайти найдовший підрядок, який уже є в словнику. Коли знайдено новий підрядок, йому присвоюється новий код і додається до словника. Тоді закодоване повідомлення є послідовністю кодів цих підрядків.

Процес декодування працює шляхом створення того самого словника символів і кодів, який використовувався для кодування. Ми зчитуємо послідовність кодів із закодованого повідомлення та використовуємо словник для перекладу кожного коду назад у відповідний символ. Коли зустрічається код, якого немає в словнику, вважається, що це код нового символу, тоді ми починаємо додавати в словник коди так, ніби ми розкодовуємо те повідомлення, яке є на даний момент. Потім знову перевіряємо, чи код є в словнику, якщо все ще немає, то вважаємо, що наступний символ збігається з тим, на якому поки що зупинились(починаючи з якого ми ще не розкодували).

In [ ]:
from time import time
class LZW:
    """
    this class allows to use LZW algorithm for encoding and decoding
    """
    def __init__(self, message, code_dict={}):
        """
        init method
        message - message that has to be encoded or decoded
        code_dict - start dictionary
        """
        self.message = message
        self.code_dict = code_dict

    def encode(self, message):
        """
        encode the message using lzw algorithm
        """
        self.message = message
        code = 0
        for elem in self.message:
            if elem not in self.code_dict:
                self.code_dict[elem] = code
                code += 1
        encoded_mes = ''
        message = self.message
        j = 0
        cur_symbol = ''
        cod_dict = self.code_dict.copy()
        length = 1
        while j+length < len(message):
            cur_symbol = message[j:j+length]
            next_symbol = message[j+length]
            line = cur_symbol + next_symbol
            length += 1
            if line not in cod_dict:
                cod_dict[line] = code
                code += 1

                encoded_mes += str(cod_dict[cur_symbol])+','
                length = 1
                j += len(line) - 1

        if length == 1:
            encoded_mes += str(cod_dict[message[-1]])
        else:
            if j + length == len(message):
                encoded_mes += str(cod_dict[cur_symbol]) + ',' + str(cod_dict[message[-1]])

        return encoded_mes

    def decode(self, message):
        """
        decode the message using lzw algorithm
        """
        self.message = message
        cod_dict1 = self.code_dict.copy()
        cod_dict2 = cod_dict1.copy()

        cod_dict2 = {v: k for k, v in cod_dict2.items()}

        encoded = self.message.split(',')
        decoded_mes = ''
        i = 0
        index = 0
        code = len(cod_dict1)

        while i < len(encoded):
            if int(encoded[i]) in cod_dict2:
                decoded_mes += cod_dict2[int(encoded[i])]
                i += 1
            else:
                length = 1
                cur_symbol = ''
                flag = False
                while index+length < len(decoded_mes):
                    flag = True
                    cur_symbol = decoded_mes[index:index+length]
                    next_symbol = decoded_mes[index+length]
                    line = cur_symbol + next_symbol
                    length += 1
                    if line not in cod_dict1:
                        cod_dict1[line] = code
                        cod_dict2[code] = line
                        code += 1
                        length = 1
                        index += len(line) - 1

                if int(encoded[i]) not in cod_dict2:
                    if flag:
                        line = decoded_mes[index:]
                        line += line[0]

                    else:
                        line = decoded_mes[-1] + decoded_mes[-1]

                    cod_dict1[line] = code
                    cod_dict2[code] = line
                    code += 1
                    index += len(line) - 1
                    length = 1

        return decoded_mes


if __name__ == "__main__":
    with open('text_example.txt', 'r', encoding='utf-8') as file:
        text = ''.join(file.readlines())

        lzw = LZW(text)
        # measure the compression time
        start = time()
        enc_mes = lzw.encode(text)
        print(f"Time of compress with a file size of {len(text)} characters:  \t\
        {time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")

        # measure the decompression time
        start = time()
        dec_mes = lzw.decode(enc_mes)

        print(f"Time of decompress with a file size of {len(text)} characters:\t\
        {time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")

        # print the percentage of file compression

        print(f'Percentage of compression with a file size of {len(text)} characters:\
        \t{len(enc_mes)/len(text.encode("utf-8"))*100}%')

        assert dec_mes == text


# 3. LZ77

Цей код є реалізацією алгоритму стиснення даних LZ77.

Клас LZ77 містить два методи - compress і decompress - які використовують алгоритм для стиснення та розпакування даних відповідно.

У методі compress текстовий файл зчитується в змінну text. Далі, передана рядок кодується у байти, тобто перетворюється на послідовність чисел, які можуть бути збережені в комп'ютері. Метод знаходить збіги між даними, які вже були зчитані, і даними, які ще мають бути зчитані. Якщо збіг знайдено, метод створює трикутник (offset, length, next_char), де offset - це позиція першого збігу в попередньому тексті, length - довжина збігу, а next_char - наступний символ, що йде після збігу. Якщо збіг не знайдено, метод додає одиничний трикутник (0, 0, next_char), де next_char - наступний символ, що має бути зчитаний. Після створення всіх трикутників, метод повертає їх як список.

У методі decompress метод приймає стиснуті дані у вигляді списку трикутників (offset, length, next_char), які були створені методом compress. Метод проходиться по списку трикутників і розкодовує їх, створюючи розгорнутий текст. Якщо length більше нуля, то в розгорнутий текст додається довжина символів, які повторюються від позиції (len(output) - offset) до length. Інакше, якщо length дорівнює нулю, то в розгорнутий текст додається наступний символ, що має бути декодований.

In [3]:
class LZ77:
    """
    This class allows you to use the LZ77 algorithmto compress text and decrypt it.
    >>> lz = LZ77()
    """
    def __init__(self, window_size=64, buffer_size=16):
        """
        Constructor of the class
        The window_size parameter specifies the size of the search window
        and the buffer_size parameter specifies the size of the input view buffer.
        (By default: window_size=1024, buffer_size=64)
        >>> lz = LZ77(2048, 32)
        """
        self.window_size = window_size
        self.buffer_size = buffer_size

    def compress(self, string):
        """
        According to the LZ77 compression algorithm, it compresses
        the file and returns a compact encrypted text in the form
        of tuples of the type (offset, length, next_char)
        >>> mini_text = "asdasdasdasdasd"
        >>> lz.compress(mini_text)
        """
        data, output, iter = string.encode(), [], 0
        while iter < len(data):
            best_match = (0, 0)
            search_start = max(0, iter - self.window_size)
            for j in range(search_start, iter):
                length = 0
                while iter + length < len(data) and data[j + length] == data[iter + length]:
                    length += 1
                    if length > best_match[1]:
                        best_match = (iter - j, length)
            if best_match[1] > 0:
                try:
                    output.append((best_match[0], best_match[1], data[iter + best_match[1]]))
                except IndexError:
                    output.append((best_match[0], best_match[1], 0))
                iter += best_match[1] + 1
            else:
                output.append((0, 0, data[iter]))
                iter += 1
        return output

    def decompress(self, compressed_data):
        """
        The decompress method takes a compressed sequence of data
        as a list of tuples and returns an expanded string. It
        restores the original data using information from the
        tuples obtained from the compress method.
        >>> lz.decompress([(0, 0, 'a'), (0, 0, 's'), (0, 0, 'd'), (3, 12, '')])
        'asdasdasdasdasd'
        """
        output = b""
        for triple in compressed_data:
            if triple[1] > 0:
                start = len(output) - triple[0]
                for i in range(triple[1]):
                    output += bytes([output[start + i]])
            output += bytes([triple[2]])
        if output[-1] == 0:
            return output[:-1].decode()# becouse 0 at end
        else: return output[:].decode()# becouse 0 at end
lz = LZ77()

# 4. Deflate

my text

In [ ]:
#my code

# Checking the serviceability of algorithms

In [4]:
mini_text = "asdasdasdasdasd"

# Try Huffman's algorithm
# напиши

# Try LZW
# напиши

# Try LZ77
lz = LZ77()
compressed = lz.compress(mini_text)
decompressed = lz.decompress(compressed)
assert mini_text == decompressed

# Try Deflate
# я напишу


# Comparison of running time and compression size in percentage of these codes

In [7]:
from time import time
lz = LZ77()


#__________________Easy_level___________________
with open('text1.txt', 'r', encoding="utf-8") as file:
    text = file.read()
#напиши Гафмана
# Напиши LZW
#LZ77
start = time()
compressed = lz.compress(text)
print(f"Time of compress LZ77 with a file size of {len(text)} characters:  \t{time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")
start = time()
decompressed = lz.decompress(compressed)
print(f"Time of decompress LZ77 with a file size of {len(text)} characters:\t{time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")
print(f'Percentage of compression LZ77 with a file size of {len(text)} characters:\t{len(compressed)*(10 + 6 + 8)/len(text.encode())/8*100}%\n')

#Huffman
huffman = Huffman(text)
# measure the compression time
start = time()
encoded_message = huffman.encode(text)
print(f"Time of compress with a file size of {len(text)} characters:  \t\
{time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")

# measure the decompression time
start = time()
decoded_message = huffman.decode(encoded_message)
print(f"Time of decompress with a file size of {len(text)} characters:\t\
{time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")


# print the percentage of file compression    
compr = (1 - (len(encoded_message) / (len(text.encode("utf-8"))*8))) * 100
#  print (f"compression: {round(compr, 2)}%")
print(f'Percentage of compression with a file size of {len(text)} characters:\
\t{compr}%')


Time of compress LZ77 with a file size of 98252 characters:  	0.8866336345672607s	= 0.01478min.(+-)
Time of decompress LZ77 with a file size of 98252 characters:	0.22539639472961426s	= 0.00376min.(+-)
Percentage of compression LZ77 with a file size of 98252 characters:	130.45575752641113%


In [8]:
# _________________Medium_level____________________
with open('text2.txt', 'r', encoding="utf-8") as file:
    text = file.read()
#напиши Гафмана
# Напиши LZW
#LZ77
start = time()
compressed = lz.compress(text)
print(f"Time of compress LZ77 with a file size of {len(text)} characters:  \t{time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")
start = time()
decompressed = lz.decompress(compressed)
print(f"Time of decompress LZ77 with a file size of {len(text)} characters:\t{time() - start}s\t= {round((time() - start)/60, 5)}min.(+-)")
print(f'Percentage of compression LZ77 with a file size of {len(text)} characters:\t{len(compressed)*(10 + 6 + 8)/len(text.encode())/8*100}%\n')

# Надто довго компілює, тому крашиться, поки не пиши тут нічого
with open('text3.txt', 'r', encoding="utf-8") as file:
    text = file.read()

Time of compress LZ77 with a file size of 528474 characters:  	4.470831632614136s	= 0.07451min.(+-)
Time of decompress LZ77 with a file size of 528474 characters:	6.080148696899414s	= 0.10134min.(+-)
Percentage of compression LZ77 with a file size of 528474 characters:	123.99487045080126%

